<a href="https://colab.research.google.com/github/MohamedAbozed/C-Users-GIS-KA-Documents-R_Data-Analysis-Data-Exploration/blob/main/Openspaces_Compliance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This script automates spatial analysis to check if 95% of residential zones are within 400 meters of Open Spaces.

In [ ]:
import arcpy  # Import ArcPy for GIS operations
import os  # To manage file paths
import datetime  # To generate report dates

In [ ]:
open_spaces = r"C:\GIS_Project\Data\OpenSpaces.gdb\OpenSpaces"
residential_zones = r"C:\GIS_Project\Data\Zoning.gdb\Residential_Zones"
road_network = r"C:\GIS_Project\Data\Roads.gdb\Roads"
aerial_imagery = r"C:\GIS_Project\Data\Imagery.gdb\LatestAerial"


In [ ]:
arcpy.RepairGeometry_management(open_spaces)
arcpy.RepairGeometry_management(residential_zones)


In [ ]:
buffer_output = os.path.join(output_folder, "OpenSpace_Buffer_400m.shp")
arcpy.Buffer_analysis(open_spaces, buffer_output, "400 Meters", "FULL", "ROUND", "ALL")


In [ ]:
overlay_output = os.path.join(output_folder, "Accessible_Residential.shp")
arcpy.Intersect_analysis([buffer_output, residential_zones], overlay_output, "ALL")


In [ ]:
total_area = sum([row[0] for row in arcpy.da.SearchCursor(residential_zones, ["SHAPE@AREA"])])
accessible_area = sum([row[0] for row in arcpy.da.SearchCursor(overlay_output, ["SHAPE@AREA"])])
compliance_percentage = (accessible_area / total_area) * 100


In [ ]:
compliance_status = "Compliant" if compliance_percentage >= 95 else "Non-Compliant"


In [ ]:
report_file = os.path.join(report_folder, "Accessibility_Report.txt")
with open(report_file, "w") as report:
    report.write(f"Date: {datetime.date.today()}\n")
    report.write(f"Total Residential Area: {total_area:.2f} sq.m\n")
    report.write(f"Accessible Area: {accessible_area:.2f} sq.m\n")
    report.write(f"Compliance Percentage: {compliance_percentage:.2f}%\n")
    report.write(f"Status: {compliance_status}\n")
